In [2]:
from pathlib import Path
import numpy as np
import torch
import torch_geometric
import torch_scatter

from torch import nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import pytorch_lightning as pl

In [5]:
path = Path("../data/synthetic")
with np.load(path/"answers.npz") as f:
    ocr = torch.Tensor(f["off_center"])
    prm = torch.Tensor(f["perimeter"])
cnn = torch.Tensor( np.load(path/"cnn.npy") )
syn = torch.Tensor( np.load(path/"synthetic.npy") )

In [8]:
prm.min()

tensor(13.7476)

# Dataset

In [3]:
def scaling(y):
    min_y = y.min() #-10.
    max_y = y.max() #10.
    return ( y - min_y ) / ( max_y - min_y )

class CNNDataModule(pl.LightningDataModule):
    def __init__(self, data_dir="../data/synthetic", task=0, num_workers=8):
        # 0: off-center, 1: perimeter 2: combined
        super().__init__()
        self.data_dir = Path(data_dir)
        self.task = task
        self.num_workers = num_workers

    def setup(self, stage=None):
        X = torch.Tensor( np.load(self.data_dir/"cnn.npy") ).permute(0,4,1,2,3)
        # X = X[:, ::2] # skip channel 1 - non-special atoms
        
        y = []
        with np.load(self.data_dir/"answers.npz") as f:
            # y.append( scaling(torch.Tensor(f["off_center"])) )
            # y.append( scaling(torch.Tensor(f["perimeter"])) )
            y.append( torch.Tensor(f["off_center"]) )
            y.append( torch.Tensor(f["perimeter"]) )
        # y.append( torch.abs(y[0]-y[1]) )
        y.append( torch.abs(scaling(y[0]) - scaling(y[1])) )

        dataset = TensorDataset(X, y[self.task])

        # Split
        full, test = random_split(dataset, [18000, 2000])

        if stage == 'fit' or stage is None:
            self.cnn_train, self.cnn_val = random_split(full, [16000, 2000])
            self.dims = tuple(self.cnn_train[0][0].shape)
        if stage == 'test' or stage is None:
            self.cnn_test = test
            self.dims = tuple(self.cnn_test[0][0].shape)
    
    def train_dataloader(self):
        return DataLoader(self.cnn_train, batch_size=32, num_workers=self.num_workers)
        
    def val_dataloader(self):
        return DataLoader(self.cnn_val, batch_size=32, num_workers=self.num_workers)

    def test_dataloader(self):
        return DataLoader(self.cnn_test, batch_size=32, num_workers=self.num_workers)

# CNN model

In [4]:
KERNEL_SIZE = 3

class ShallowCNN(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv3d(3, 32, (KERNEL_SIZE,KERNEL_SIZE,KERNEL_SIZE))
        self.conv2 = nn.Conv3d(32, 32, (KERNEL_SIZE,KERNEL_SIZE,KERNEL_SIZE)) 
        self.conv3 = nn.Conv3d(32, 32, (KERNEL_SIZE,KERNEL_SIZE,KERNEL_SIZE)) 
        self.fc = nn.Linear(32, 1)
    
    def forward(self, x):
        out = F.relu( self.conv1(x) )
        out = F.relu( self.conv2(out) )
        out = F.relu( self.conv3(out) )
        out = F.adaptive_max_pool3d(out, (1,1,1))
        out = out.view(out.shape[0], -1)
        out = self.fc(out)
        return out

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

    def training_step(self, batch, batch_idx):
        x, y = batch
        # y = scaling(y).unsqueeze(1)
        y = y.unsqueeze(1)
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y)
        self.log("train_loss", loss, on_epoch=True, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        # y = scaling(y).unsqueeze(1)
        y = y.unsqueeze(1)
        y_hat = self(x)
        val_loss = F.mse_loss(y_hat, y)
        self.log("val_loss", val_loss, on_epoch=True, prog_bar=True)
        return val_loss
    
    def test_step(self, batch, batch_idx):
        x, y = batch
        # y = scaling(y).unsqueeze(1)
        y = y.unsqueeze(1)
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y)
        self.log("test_loss", loss, on_epoch=True, prog_bar=True)
        return loss

# Check number of trainable parameters
model = ShallowCNN()
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
pytorch_total_params

58017

In [4]:
task = 1

dm = CNNDataModule(task=task)
model = ShallowCNN()

trainer = pl.Trainer(max_epochs=40)
trainer.fit(model, dm)
trainer.test(datamodule=dm)
# To see the plots, on the terminal run: tensorboard --logdir ./lightning_logs


GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name  | Type   | Params
---------------------------------
0 | conv1 | Conv3d | 2.6 K 
1 | conv2 | Conv3d | 27.7 K
2 | conv3 | Conv3d | 27.7 K
3 | fc    | Linear | 33    
---------------------------------
58.0 K    Trainable params
0         Non-trainable params
58.0 K    Total params
0.232     Total estimated model params size (MB)
Epoch 0:   5%|▌         | 30/563 [00:04<01:12,  7.34it/s, loss=1.54e+03, v_num=19, val_loss=4.02e+3, train_loss_step=694.0]
/home/fiqah_kmy99_gmail_com/miniconda3/envs/gvp/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)
Testing: 100%|██████████| 63/63 [00:02<00:00, 21.49it/s]
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 499.7353820800781}
------------------------------------

[{'test_loss': 499.7353820800781}]

# Resume training from checkpoint

In [6]:
task = 1

dm = CNNDataModule(task=task)
model = ShallowCNN()

trainer = pl.Trainer(resume_from_checkpoint="./lightning_logs/version_18/checkpoints/epoch=39-step=19999.ckpt", max_epochs=100)
trainer.fit(model, dm)
trainer.test(datamodule=dm)

n_loss_step=41.90, train_loss_epoch=42.30]
Epoch 94:  89%|████████▉ | 501/563 [00:57<00:07,  8.78it/s, loss=60.2, v_num=19, val_loss=80.10, train_loss_step=42.10, train_loss_epoch=42.30]
Validating: 0it [00:00, ?it/s]
Epoch 95:  89%|████████▉ | 501/563 [00:56<00:07,  8.81it/s, loss=59, v_num=19, val_loss=86.20, train_loss_step=40.40, train_loss_epoch=43.40]
Validating: 0it [00:00, ?it/s]
Epoch 96:  89%|████████▉ | 501/563 [00:56<00:07,  8.82it/s, loss=56.3, v_num=19, val_loss=96.90, train_loss_step=38.20, train_loss_epoch=42.70]
Validating: 0it [00:00, ?it/s]
Epoch 97:  89%|████████▉ | 501/563 [00:56<00:07,  8.82it/s, loss=55, v_num=19, val_loss=102.0, train_loss_step=36.60, train_loss_epoch=41.40]
Validating: 0it [00:00, ?it/s]
Epoch 98:  89%|████████▉ | 501/563 [00:57<00:07,  8.74it/s, loss=52.2, v_num=19, val_loss=104.0, train_loss_step=35.30, train_loss_epoch=40.30]
Validating: 0it [00:00, ?it/s]
Epoch 99:  89%|████████▉ | 501/563 [00:56<00:07,  8.80it/s, loss=50.6, v_num=19, val_l

[{'test_loss': 63.581539154052734}]

# Loading from checkpoint

In [4]:
task = 1

dm = CNNDataModule(task=task)
model = ShallowCNN.load_from_checkpoint("./lightning_logs/version_13/checkpoints/epoch=14-step=7499.ckpt")
model.eval()

trainer = pl.Trainer()
trainer.test(model, datamodule=dm)



GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name  | Type   | Params
---------------------------------
0 | conv1 | Conv3d | 49.2 K
1 | fc    | Linear | 33    
---------------------------------
49.2 K    Trainable params
0         Non-trainable params
49.2 K    Total params
0.197     Total estimated model params size (MB)
Epoch 0:  89%|████████▉ | 500/563 [00:22<00:02, 22.67it/s, loss=167, v_num=14, val_loss=153.0, train_loss_step=114.0]
Validating: 0it [00:00, ?it/s]
Epoch 1:  89%|████████▉ | 500/563 [00:21<00:02, 23.20it/s, loss=157, v_num=14, val_loss=157.0, train_loss_step=108.0, train_loss_epoch=159.0]
Validating: 0it [00:00, ?it/s]
Epoch 2:  89%|████████▉ | 500/563 [00:21<00:02, 23.38it/s, loss=147, v_num=14, val_loss=151.0, train_loss_step=101.0, train_loss_epoch=150.0]
Validating: 0it [00:00, ?it/s]
Epoch 3:  89%|████████▉ | 500/563 [00:21<00:02, 23.53it/s, loss=138, v_num=14, val_loss=145.0, train_loss_step=94.20, train_loss_epoch=142.0]
Valid

[{'test_loss': 4077.56591796875}]